In [1]:
%load_ext autoreload
%autoreload 2
from qiskit_metal import draw, Dict, designs, MetalGUI
from qiskit_metal.toolbox_metal import math_and_overrides
from qiskit_metal.qlibrary.core import QComponent
import qiskit_metal as metal
from just_claw import TransmonClaw
# ?metal.qlibrary.core.QComponent.add_qgeometry

design = metal.designs.design_planar.DesignPlanar()
gui = metal.MetalGUI(design)
design.overwrite_enabled = True

# dir(QComponent)
cpw_width = '11.7um'

design._chips.main.size.size_x = '5mm'
design._chips.main.size.size_y = '5mm'
from qiskit_metal.qlibrary.terminations.launchpad_wb import LaunchpadWirebond
from qiskit_metal.qlibrary.terminations.short_to_ground import ShortToGround
from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight
from qiskit_metal.qlibrary.tlines.anchored_path import RouteAnchors
from qiskit_metal.qlibrary.tlines.mixed_path import RouteMixed
from qiskit_metal.qlibrary.qubits.transmon_cross import TransmonCross
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.terminations.open_to_ground import OpenToGround
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee
from qiskit_metal.qlibrary.couplers.cap_n_interdigital_tee import CapNInterdigitalTee
from qiskit_metal.qlibrary.couplers.line_tee import LineTee

from collections import OrderedDict

import numpy as np

xmon1_options = dict(
    connection_pads = dict(
        readout = dict(
            connector_location='90',
            connector_type='0',
            claw_length="150um",
            ground_spacing='4.1um',
            claw_gap='5.1um',
            claw_width='15um',
            claw_cpw_length=0
        ),
    ),
    cross_width='30um',
    cross_length="100um",
    cross_gap='30um',
    orientation='-90',
    pos_x='-1500um',
    pos_y='1200um',
    # aedt_hfss_inductance=inductance,
    # hfss_inductance=inductance, 
    # q3d_inductance=inductance,
    # aedt_q3d_inductance=inductance
)

# Create a new Transmon Cross object with name 'Q1'
q = TransmonClaw(design, 'Q', options=xmon1_options)

clt1_opts = Dict(prime_width = cpw_width,
                prime_gap = '5.1um',
                second_width = cpw_width,
                second_gap = '5.1um',
                coupling_space = '7.9um',
                coupling_length = '223.5um',
                open_termination = False,
                orientation = '-90',
                pos_y = '1200um',
                down_length = '50um')

clt = CoupledLineTee(design, 'clt', clt1_opts)

cpw_total_length = 5160 + 716 + 300 + 1881.18568
cpw_total_length = str(cpw_total_length) + 'um'
cpw1_opts = Dict(pin_inputs = Dict(start_pin = Dict(component = 'clt',
                                                    pin = 'second_end'),
                                   end_pin = Dict(component = 'Q',
                                                  pin = 'readout')),
                lead = Dict(end_straight = '70um',
                            # start_jogged_extension = jogsS),
                ),
                fillet = '49.9um',
                total_length = '3885.5930858724796um',#'3893.8166um', # '6776um',
                trace_width = cpw_width,
                meander = Dict(spacing = '100um',
                               asymmetry = '-150um'),
                trace_gap = '5.1um',)
cpw = RouteMeander(design, 'cpw', options = cpw1_opts)

gui.rebuild()
gui.autoscale()
# gui.zoom_on_components(['clt5', 'Q5'])
# gui.screenshot()

In [2]:
predicted_cross_length = [334.2132054053733, 184.22335346133133, 246.93411358962248, 245.543257860049, 185.85846413905253]
predicted_claw_length = [150.0, 160.0, 230.0, 180.0, 170.0]
Lj_vals = [7.787523742211472, 13.718572219699162, 10.60479253401587, 10.239078841902765, 13.486696492020988]
cpw_lengths = [3885.5930858724796, 3729.2684730129495, 4262.170190739973, 3726.340253625585, 3860.5768979267004]
cplr_lengths = [223.99034604652442, 211.31821443396487, 218.27779609919682, 225.65194847225646, 225.28882915922742]

# Eigenmode Sim (CLT + CPW + Claw)

In [3]:
from qiskit_metal.analyses.quantization import EPRanalysis
import pyEPR as epr
from pyaedt import Hfss
from utils import *
from objects import *
from datetime import datetime
import pprint

In [4]:
eig_q12bus = EPRanalysis(design, "hfss")

In [5]:
def do_eigenmode(claw_length, Lj, index):                
    design.components['Q'].options['connection_pads']['readout']['claw_length'] = f'{claw_length}um'
    design.components['Q'].options['hfss_inductance'] = Lj
    design.components['Q'].options['aedt_hfss_inductance'] = Lj
    design.components['Q'].options['q3d_inductance'] = Lj
    design.components['Q'].options['aedt_q3d_inductance'] = Lj
    design.rebuild()

    eig_q12bus = EPRanalysis(design, "hfss")
    hfss = eig_q12bus.sim.renderer
    hfss.start()

    hfss.new_ansys_design(f"Cavity_Claw{index}", 'eigenmode')
    hfss.render_design([qubit_name, feedline_name, cavity_name])  # List of components to include in the simulation

    eig_q12bus.sim.setup.max_passes = 50
    eig_q12bus.sim.setup.max_delta_f = 0.1
    eig_q12bus.sim.setup.n_modes = 2

    sim_setup_var_dict = dict()

    key_Lj = 'Lj'
    key_Cj = 'Cj'
    sim_setup_var_dict[key_Lj] = str(design.components['Q'].options['hfss_inductance']) + ' H'
    sim_setup_var_dict[key_Cj] = str(design.components['Q'].options['hfss_capacitance']) + ' F'
    eig_q12bus.sim.setup.vars = sim_setup_var_dict

    gui.rebuild()
    gui.autoscale()

    # aedt = Hfss(projectname=hfss.pinfo.project_name, 
    #             designname=hfss.pinfo.design.name, 
    #             solution_type='eigenmode',
    #             new_desktop_session=False, 
    #             close_on_exit=False)
    # materials = aedt.materials
    # silicon = materials.checkifmaterialexists('silicon')
    # silicon.permittivity = 11.45
    # silicon.dielectric_loss_tangent = 1E-7
    
    qubit_name = "Q"
    feedline_name = "cpw"
    cavity_name = "clt"
    mesh_lengths = {'mesh1': {"objects": [f"cross_{'qubit_name'}",f"readout_connector_arm_{qubit_name}",f"prime_cpw_{feedline_name}", f"second_cpw_{feedline_name}", f"trace_{cavity_name}", f"readout_connector_arm_{feedline_name}"], "MaxLength": '4um'},
                    'mesh2': {'objects': [f"JJ_rect_Lj_{qubit_name}_rect_jj"], "MaxLength": '2um'}
                    }
    add_ground_strip_and_mesh(hfss.pinfo.design.modeler, clt, mesh_lengths)


    eig_q12bus.sim.run(name='full_device_v1.0',
                    components=['Q', 'cpw', 'clt'],
                    open_terminations = [('clt', 'prime_start'), ('clt', 'prime_end')],
                    port_list = [('clt', 'prime_start', '50'), ('clt', 'prime_end', '50')]
                    )
    
    f = eig_q12bus.get_frequencies()

    freq = f.values[0][0] * 1e9 # in linear Hz
    Q = f.values[0][1]
    kappa_emode = {round(freq/Q/1e6, 3)} # in linear MHz

    return f,Q,kappa_emode

In [6]:
print(round(cplr_lengths[0], 10))

223.9903460465


In [7]:
%matplotlib inline
eig_q12bus = EPRanalysis(design, "hfss")
hfss = eig_q12bus.sim.renderer

# i = 4
# setups, hfsss = [], []

# do_eigenmode(predicted_claw_length[1], Lj_vals[1])



for i, (cross_length, claw_length, Lj, cpw_len, cplr_len) in enumerate(zip(predicted_cross_length, predicted_claw_length, Lj_vals, cpw_lengths, cplr_lengths)):
    if i != 0:
        break

    xmon1_options["connection_pads"]["readout"]["claw_length"] = f"{claw_length}um"
    xmon1_options["aedt_hfss_inductance"]=Lj,
    xmon1_options["hfss_inductance"]=Lj, 
    xmon1_options["q3d_inductance"]=Lj,
    xmon1_options["aedt_q3d_inductance"]=Lj

    cpw1_opts["total_length"] = f"{cpw_len}um"
    clt1_opts["coupling_length"] = f"{int(round(cplr_len, 0))}um"
    
    v2_opts = Dict(
        claw_opts = xmon1_options,
        cplr_opts = clt1_opts,
        cpw_opts = cpw1_opts
    )
    # pprint.pprint(v2_opts)
    now = datetime.now()
    CLT_epr_sweep(design = design, sweep_opts=v2_opts, filename = f"cavity_claw_num{i}_{now.strftime('%d_%m_%Y_%H_%M_%S')}_v2")


INFO 07:21PM [connect_project]: Connecting to Ansys Desktop API...


Starting the Simulation


INFO 07:21PM [load_ansys_project]: 	Opened Ansys App
INFO 07:21PM [load_ansys_project]: 	Opened Ansys Desktop v2021.1.0
INFO 07:21PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    d:/andre/paper_sweeps/project_storage/
	Project:   clt_sweep_3970um_220um
INFO 07:21PM [connect_design]: 	Opened active design
	Design:    CavitySweep_hfss [Solution type: Eigenmode]
INFO 07:21PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)
INFO 07:21PM [connect]: 	Connected to project "clt_sweep_3970um_220um" and design "CavitySweep_hfss" 😀 

INFO 07:21PM [connect_design]: 	Opened active design
	Design:    CavitySweep16 [Solution type: Eigenmode]
WARNING 07:21PM [connect_setup]: 	No design setup detected.
WARNING 07:21PM [connect_setup]: 	Creating eigenmode default setup.
INFO 07:21PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)
INFO 07:21PM [connect_design]: 	Opened active design
	Design:    CavitySweep_hfss [Solution type: Eigenmode]


Sim rendered into HFSS!
pyaedt INFO: using existing logger.


INFO:Global:using existing logger.


...

ERROR:Global:Arguments with values: 


pyaedt ERROR: Check Online documentation on: https://aedt.docs.pyansys.com/search.html?q=_refresh_planes


ERROR:Global:Check Online documentation on: https://aedt.docs.pyansys.com/search.html?q=_refresh_planes


pyaedt INFO: Project files removed from handlers.


INFO:Global:Project files removed from handlers.
INFO 07:22PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)
INFO 07:22PM [analyze]: Analyzing setup Setup


# LOM Analysis

In [ ]:
xmon = TransmonCross(design = design, options = xmon1_options, name = "xmon")

In [ ]:
from qiskit_metal.analyses.quantization import LOManalysis
c1 = LOManalysis(design, "q3d")

In [ ]:
from qiskit_metal.analyses.quantization import LOManalysis
c1 = LOManalysis(design, "q3d")

In [ ]:
c1.sim.setup.reuse_selected_design = False
c1.sim.setup.reuse_setup = False

# example: update single setting
c1.sim.setup.max_passes = 30
c1.sim.setup.min_converged_passes = 1
c1.sim.setup.percent_error = 0.1
c1.sim.setup.name = 'sweep_setup'

In [ ]:
def do_LOM_qubitonly(cross_length, claw_length, Lj, i):
    qname = 'xmon'
    cname = 'readout'

    design.components[qname].options['connection_pads']['readout']['claw_length'] = f'{claw_length}um'
    design.components[qname].options['cross_length'] = f'{cross_length}um'
    design.components[qname].options['hfss_inductance'] = Lj
    design.components[qname].options['aedt_hfss_inductance'] = Lj
    design.components[qname].options['q3d_inductance'] = Lj
    design.components[qname].options['aedt_q3d_inductance'] = Lj
    design.rebuild()

    selection = [qname]
    open_pins = [(qname, cname)]


    c1.sim.run(name = 'LOMv2.0', components=selection,
               open_terminations=open_pins)
    cap_df = c1.sim.capacitance_matrix

    print(f'For Qubit_{i}, the cap matrix is...')
    print(cap_df)

    data = {
        "design_options": design.components[qname].options,
        "data": {
            "cross_to_ground": cap_df.loc[f'cross_{qname}']['ground_main_plane'],
            "claw_to_ground": cap_df.loc[f'{cname}_connector_arm_{qname}']['ground_main_plane'],
            "cross_to_claw": cap_df.loc[f'cross_{qname}'][f'{cname}_connector_arm_{qname}'],
            "cross_to_cross": cap_df.loc[f'cross_{qname}']['ground_main_plane'],
            "claw_to_claw": cap_df.loc[f'{cname}_connector_arm_{qname}'][f'{cname}_connector_arm_{qname}'],
            "ground_to_ground": cap_df.loc['ground_main_plane']['ground_main_plane']
        },
        "sim_info": {
            "setup": c1.sim.setup,
            "renderer_options": c1.sim.renderer.options
        }
    }
    save_simulation_data_to_json(data, filename = f"qubitonly_{i}")


In [ ]:
all_cap_matricies = []

for i, (cross_length, claw_length, Lj) in enumerate(zip(predicted_cross_length, predicted_claw_length, Lj_vals)):
    if i == 1:
        continue
    do_LOM_qubitonly(cross_length, claw_length, Lj, i)